# Address Segmentation
Conversion of address points into segmented address ranges along a road network.

**Notes:** The following guide assumes data has already been preprocessed including data scrubbing and filtering.

In [1]:
import contextily as ctx
import geopandas as gpd
import ipympl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import shapely
from bisect import bisect
from collections import OrderedDict
from operator import itemgetter
from shapely.geometry import LineString, Point

## Load dataframes and configure attributes
Loads dataframes into geopandas and separates address numbers and suffixes, if required.

In [2]:
# Load dataframes.
addresses = gpd.read_file("C:/scratch/City_Of_Yellowknife.gpkg", layer="addresses")
roads = gpd.read_file("C:/scratch/City_Of_Yellowknife.gpkg", layer="roads")

# Configure attributes - number and suffix.
addresses["suffix"] = addresses["number"].map(lambda val: re.sub(pattern="\\d+", repl="", string=val, flags=re.I))
addresses["number"] = addresses["number"].map(lambda val: re.sub(pattern="[^\\d]", repl="", string=val, flags=re.I)).map(int)

addresses.head()

,number,street,geometry,suffix
0,43,OTTO DRIVE,POINT Z (-12728491.168 8972072.805 0.000),A
1,48,TAYLOR ROAD,POINT Z (-12733829.550 8964622.287 0.000),
2,136,DE WEERDT DRIVE,POINT Z (-12731620.348 8969989.275 0.000),B
3,129,HAENER DRIVE,POINT Z (-12732186.396 8969861.224 0.000),B
4,2,STIRLING COURT,POINT Z (-12731633.895 8970568.762 0.000),B


In [3]:
roads.head()

,featurecod,community,routename,surface,date,roadname,geometry
0,RSTRLIN,Northwest Territories,None,Unpaved,2015-09-28,VEE LAKE ROAD,"LINESTRING Z (-12731561.088 8988790.623 0.000,..."
1,RRESLIN,Northwest Territories,None,Unpaved,2015-09-28,None,"LINESTRING Z (-12731215.603 8990017.829 0.000,..."
2,RHWYLIN,Northwest Territories,Highway 4,Paved,2015-09-28,INGRAHAM TRAIL,"LINESTRING Z (-12723960.099 8980833.256 0.000,..."
3,RARTLIN,Northwest Territories,None,Unpaved,2015-09-28,DETTAH ROAD,"LINESTRING Z (-12723909.403 8972227.140 0.000,..."
4,RSTRLIN,Northwest Territories,None,Unpaved,2015-09-28,DETTAH,"LINESTRING Z (-12724863.590 8957290.788 0.000,..."


## Preview data

In [4]:
%matplotlib widget

# Fetch basemap.
# Note: basemaps are retrieved in EPSG:3857 and, therefore, dataframes should also use this crs.
basemap, extent = ctx.bounds2img(*roads.total_bounds, ll=False, source=ctx.providers.Esri.WorldImagery)

# Configure plot.
fig, ax = plt.subplots(tight_layout=True)
plt.imshow(basemap, extent=extent)
addresses.plot(ax=ax, color="red", label="addresses", markersize=3)
roads.plot(ax=ax, color="cyan", label="roads", linewidth=1)
ax.ticklabel_format(style="plain")
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha="right")
plt.xlabel("Longitude (m)")
plt.ylabel("Latitude (m)")
plt.title("City of Yellowknife")
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Configure address to roads linkages
Links addresses to the nearest, matching road segment.

In [5]:
# Link addresses and roads on join fields.
addresses["road_index"] = addresses["street"].map(lambda val: tuple(set(roads[roads["roadname"] == val].index)))
addresses.head()

,number,street,geometry,suffix,road_index
0,43,OTTO DRIVE,POINT Z (-12728491.168 8972072.805 0.000),A,"(389, 390, 391, 361, 911, 370)"
1,48,TAYLOR ROAD,POINT Z (-12733829.550 8964622.287 0.000),,"(1382, 1383, 168, 170, 651, 650, 173, 652, 654..."
2,136,DE WEERDT DRIVE,POINT Z (-12731620.348 8969989.275 0.000),B,"(804, 807)"
3,129,HAENER DRIVE,POINT Z (-12732186.396 8969861.224 0.000),B,"(800, 801, 802, 803, 806, 338)"
4,2,STIRLING COURT,POINT Z (-12731633.895 8970568.762 0.000),B,"(342,)"


In [6]:
# Reduce linkages to one road index per address.

def get_nearest_linkage(pt, road_indexes):
    """Returns the road index associated with the nearest road geometry to the given address point."""
    
    # Get road geometries.
    road_geometries = tuple(map(lambda index: roads["geometry"].iloc[index], road_indexes))
    
    # Get road distances from address point.
    road_distances = tuple(map(lambda road: pt.distance(road), road_geometries))
    
    # Get the road index associated with the smallest distance.
    road_index = road_indexes[road_distances.index(min(road_distances))]
    
    return road_index


# Flag plural linkages.
flag_plural = addresses["road_index"].map(len) > 1

# Reduce plural linkages to the road with the lowest (nearest) geometric distance.
addresses.loc[flag_plural, "road_index"] = addresses[flag_plural][["geometry", "road_index"]].apply(
    lambda row: get_nearest_linkage(*row), axis=1)

# Unpack first tuple element for singular linkages.
addresses.loc[~flag_plural, "road_index"] = addresses[~flag_plural]["road_index"].map(itemgetter(0))

# Compile linked road geometry for each address.
addresses["road_geometry"] = addresses.merge(
    roads["geometry"], how="left", left_on="road_index", right_index=True)["geometry_y"]

addresses.head()

,number,street,geometry,suffix,road_index,road_geometry
0,43,OTTO DRIVE,POINT Z (-12728491.168 8972072.805 0.000),A,390,"LINESTRING Z (-12728785.212 8972077.288 0.000,..."
1,48,TAYLOR ROAD,POINT Z (-12733829.550 8964622.287 0.000),,168,"LINESTRING Z (-12733834.171 8964493.879 0.000,..."
2,136,DE WEERDT DRIVE,POINT Z (-12731620.348 8969989.275 0.000),B,807,"LINESTRING Z (-12731747.194 8969855.676 0.000,..."
3,129,HAENER DRIVE,POINT Z (-12732186.396 8969861.224 0.000),B,338,"LINESTRING Z (-12732196.969 8969779.397 0.000,..."
4,2,STIRLING COURT,POINT Z (-12731633.895 8970568.762 0.000),B,342,"LINESTRING Z (-12731536.519 8970569.562 0.000,..."


## Configure address parity
Computes address-road parity (left / right side).

In [37]:
def get_parity(pt, vector):
    """
    Determines the parity (left or right side) of an address point relative to a road vector.

    Parity is derived from the determinant of the vectors formed by the road segment and the address-to-road
    vectors. A positive determinant indicates 'left' parity and negative determinant indicates 'right' parity.
    """
    
    det = (vector[1][0] - vector[0][0]) * (pt.y - vector[0][1]) - \
          (vector[1][1] - vector[0][1]) * (pt.x - vector[0][0])
    sign = np.sign(det)
    
    return "l" if sign == 1 else "r"

def get_road_vector(pt, segment):
    """
    Returns the following:
    a) the distance of the address intersection along the road segment.
    b) the vector comprised of the road segment coordinates immediately before and after the address
    intersection point.
    """
    
    # For all road segment points and the intersection point, calculate the distance along the road segment.
    # Note: always use the length as the distance for the last point to avoid distance=0 for looped roads.
    node_distance = (*map(lambda coord: segment.project(Point(coord)), segment.coords[:-1]), segment.length)
    intersection_distance = segment.project(pt)
    
    # Compute the index of the intersection point within the road points, based on distances.
    intersection_index = bisect(node_distance, intersection_distance)
    
    # Conditionally compile the road points, as a vector, immediately bounding the intersection point.
    
    # Intersection matches a pre-existing road point.
    if intersection_distance in node_distance:
        
        # Intersection matches the first road point.
        if intersection_index == 1:
            vector = itemgetter(intersection_index - 1, intersection_index)(segment.coords)
        
        # Intersection matches the last road point.
        elif intersection_index == len(node_distance):
            vector = itemgetter(intersection_index - 2, intersection_index - 1)(segment.coords)
        
        # Intersection matches an interior road point.
        else:
            vector = itemgetter(intersection_index - 2, intersection_index)(segment.coords)
    
    # Intersection matches no pre-existing road point.
    else:
        vector = itemgetter(intersection_index - 1, intersection_index)(segment.coords)
    
    return intersection_distance, vector

# Get point of intersection between each address and the linked road segment.
addresses["intersection"] = addresses[["geometry", "road_geometry"]].apply(
    lambda row: itemgetter(-1)(shapely.ops.nearest_points(*row)), axis=1)

# Get the following:
# a) the distance of the intersection point along the linked road segment.
# b) the road segment vector which bounds the intersection point.
#    i.e. vector formed by the coordinates immediately before and after the intersection point.
results = addresses[["intersection", "road_geometry"]].apply(lambda row: get_road_vector(*row), axis=1)
addresses["distance"] = results.map(itemgetter(0))
addresses["road_vector"] = results.map(itemgetter(1))

# Get address parity.
addresses["parity"] = addresses[["geometry", "road_vector"]].apply(
    lambda row: get_parity(*row), axis=1)

addresses[["geometry", "road_geometry", "intersection", "distance", "road_vector", "parity"]].head()

,geometry,road_geometry,intersection,distance,road_vector,parity
0,POINT Z (-12728491.168 8972072.805 0.000),"LINESTRING Z (-12728785.212 8972077.288 0.000,...",POINT (-12728635.1048 8971969.687900001),186.255702,"((-12728647.616799997, 8971984.796699999, 0.0)...",l
1,POINT Z (-12733829.550 8964622.287 0.000),"LINESTRING Z (-12733834.171 8964493.879 0.000,...",POINT (-12733809.82087993 8964497.738158945),24.653609,"((-12733820.3273, 8964496.073899997, 0.0), (-1...",l
2,POINT Z (-12731620.348 8969989.275 0.000),"LINESTRING Z (-12731747.194 8969855.676 0.000,...",POINT (-12731628.7192 8969920.246899998),135.547132,"((-12731657.712499999, 8969910.514499994, 0.0)...",l
3,POINT Z (-12732186.396 8969861.224 0.000),"LINESTRING Z (-12732196.969 8969779.397 0.000,...",POINT (-12732219.69627087 8969849.68197122),73.873501,"((-12732213.9375, 8969833.066699998, 0.0), (-1...",r
4,POINT Z (-12731633.895 8970568.762 0.000),"LINESTRING Z (-12731536.519 8970569.562 0.000,...",POINT (-12731619.42874114 8970534.401334839),90.057500,"((-12731587.3267, 8970547.916899998, 0.0), (-1...",r


In [39]:
df=addresses.loc[addresses.road_index==264]
df.sort_values(by='number').head(n=30)

,number,street,geometry,suffix,road_index,road_geometry,intersection,distance,road_vector,parity
3909,337,OLD AIRPORT ROAD,POINT Z (-12735998.087 8967989.639 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12735926.60848456 8968028.303715),106.060976,"((-12735923.679699998, 8968022.889399996, 0.0)...",l
3910,338,OLD AIRPORT ROAD,POINT Z (-12735894.905 8968069.295 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12735936.58569764 8968046.748149302),127.031001,"((-12735923.679699998, 8968022.889399996, 0.0)...",r
3901,339,OLD AIRPORT ROAD,POINT Z (-12736057.955 8968102.491 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12735987.38490363 8968140.665219631),233.806352,"((-12735983.8746, 8968134.175999995, 0.0), (-1...",l
3907,340,OLD AIRPORT ROAD,POINT Z (-12735943.741 8968172.519 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12735990.83525806 8968147.04361592),241.058171,"((-12735983.8746, 8968134.175999995, 0.0), (-1...",r
5762,341,OLD AIRPORT ROAD,POINT Z (-12736203.082 8968128.995 0.000),E,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736031.32193201 8968221.890285356),326.153381,"((-12736024.008499999, 8968208.367999997, 0.0)...",l
3900,341,OLD AIRPORT ROAD,POINT Z (-12736128.743 8968239.178 0.000),A,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736060.60906969 8968276.034837753),387.711231,"((-12736044.071499998, 8968245.463799996, 0.0)...",l
5760,341,OLD AIRPORT ROAD,POINT Z (-12736155.159 8968153.164 0.000),C,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736030.58981425 8968220.536624692),324.614422,"((-12736024.008499999, 8968208.367999997, 0.0)...",l
5359,341,OLD AIRPORT ROAD,POINT Z (-12736066.036 8968196.159 0.000),B,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736028.41107621 8968216.508212652),320.034574,"((-12736024.008499999, 8968208.367999997, 0.0)...",l
5761,341,OLD AIRPORT ROAD,POINT Z (-12736176.866 8968141.682 0.000),D,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736030.69754042 8968220.735806683),324.840869,"((-12736024.008499999, 8968208.367999997, 0.0)...",l
3908,342,OLD AIRPORT ROAD,POINT Z (-12735967.363 8968249.722 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736028.49307011 8968216.659816574),320.206931,"((-12736024.008499999, 8968208.367999997, 0.0)...",r


In [36]:
df=addresses.loc[addresses.road_index==264]
df.head(n=30)
x=df.loc[df.index==3901]["geometry"].iloc[0].x
y=df.loc[df.index==3901]["geometry"].iloc[0].y
vector=df.loc[df.index==3901]["road_vector"].iloc[0]
det = (vector[1][0] - vector[0][0]) * (y - vector[0][1]) - (vector[1][1] - vector[0][1]) * (x - vector[0][0])
print("{0:.10f}".format(det))
print(np.sign(det))

,number,street,geometry,suffix,road_index,road_geometry,intersection,distance,road_vector,parity
481,349,OLD AIRPORT ROAD,POINT Z (-12736359.385 8968636.951 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736279.28078767 8968680.278633779),847.309281,"((-12736264.8132, 8968653.5308, 0.0), (-127362...",l
3897,344,OLD AIRPORT ROAD,POINT Z (-12736013.716 8968292.572 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736056.91399961 8968269.204201279),379.945207,"((-12736044.071499998, 8968245.463799996, 0.0)...",r
3898,346,OLD AIRPORT ROAD,POINT Z (-12736074.241 8968394.765 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736113.38303967 8968373.594209993),498.629774,"((-12736104.274599997, 8968356.753499996, 0.0)...",r
3899,343,OLD AIRPORT ROAD,POINT Z (-12736176.298 8968317.705 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736104.23843192 8968356.68664117),479.407658,"((-12736084.206699999, 8968319.656800002, 0.0)...",l
3900,341,OLD AIRPORT ROAD,POINT Z (-12736128.743 8968239.178 0.000),A,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736060.60906969 8968276.034837753),387.711231,"((-12736044.071499998, 8968245.463799996, 0.0)...",l
3901,339,OLD AIRPORT ROAD,POINT Z (-12736057.955 8968102.491 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12735987.38490363 8968140.665219631),233.806352,"((-12735983.8746, 8968134.175999995, 0.0), (-1...",r
3902,350,OLD AIRPORT ROAD,POINT Z (-12736192.762 8968622.964 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736235.71110126 8968599.733559886),755.735122,"((-12736224.6783, 8968579.335699998, 0.0), (-1...",l
3903,352,OLD AIRPORT ROAD,POINT Z (-12736237.335 8968692.347 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736274.83673459 8968672.062420016),837.968201,"((-12736264.8132, 8968653.5308, 0.0), (-127362...",l
3904,345,OLD AIRPORT ROAD,POINT Z (-12736220.960 8968407.331 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736151.85082356 8968444.709801896),579.482713,"((-12736144.4069, 8968430.946799995, 0.0), (-1...",r
3905,347,OLD AIRPORT ROAD,POINT Z (-12736279.859 8968526.016 0.000),,264,"LINESTRING Z (-12735876.468 8967934.843 0.000,...",POINT (-12736214.8570071 8968561.181334218),711.903999,"((-12736204.608999997, 8968542.2382, 0.0), (-1...",l


## View relationship between parity inputs
View the relationship between address points, bounding road vectors, and address-road intersection points.

In [8]:
# Create geometries for viewing.
bounding_road_vectors = gpd.GeoDataFrame(geometry=addresses["road_vector"].map(LineString), crs=addresses.crs)
intersection_pt_linkages = gpd.GeoDataFrame(geometry=addresses[["geometry", "intersection"]].apply(
    lambda row: LineString([pt.coords[0][:2] for pt in row]), axis=1), crs=addresses.crs)

# Configure plot.
fix, ax = plt.subplots(tight_layout=True)
plt.imshow(basemap, extent=extent)
addresses.plot(ax=ax, color="red", label="addresses", markersize=3)
intersection_pt_linkages.plot(ax=ax, color="yellow", label="addresses - intersection point linkages", linewidth=1)
bounding_road_vectors.plot(ax=ax, color="cyan", label="bounding road vectors", linewidth=1)
ax.ticklabel_format(style="plain")
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha="right")
plt.xlabel("Longitude (m)")
plt.ylabel("Latitude (m)")
plt.title("City of Yellowknife")
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Configure address ranges (addrange) and attributes
Groups addresses into ranges by road linkage and parity, then computes the range attributes.

In [9]:
def get_digdirfg(sequence):
    """Returns the digdirfg attribute for the given sequence of address numbers."""

    sequence = list(sequence)

    # Return digitizing direction for single addresses.
    if len(sequence) == 1:
        return "Not Applicable"

    # Derive digitizing direction from sequence sorting direction.
    if sequence == sorted(sequence):
        return "Same Direction"
    else:
        return "Opposite Direction"

def get_hnumstr(sequence):
    """Returns the hnumstr attribute for the given sequence of address numbers."""

    sequence = list(sequence)

    # Validate structure for single addresses.
    if len(sequence) == 1:
        return "Even" if (sequence[0] % 2 == 0) else "Odd"

    # Configure sequence sort status.
    if sequence == sorted(sequence) or sequence == sorted(sequence, reverse=True):

        # Configure sequence parities.
        parities = tuple(map(lambda number: number % 2 == 0, sequence))

        # Validate structure for sorted address ranges.
        if all(parities):
            return "Even"
        elif not any(parities):
            return "Odd"
        else:
            return "Mixed"

    # Return structure for unsorted address ranges.
    else:
        return "Irregular"

def get_number_sequence(addresses):
    """Returns the filtered number sequence for the given addresses."""

    # Separate address components.
    numbers, suffixes, distances = tuple(zip(*addresses))

    # Reduce addresses at a duplicated intersection distance to only the first instance.
    if len(distances) == len(set(distances)):
        sequence = numbers
    else:
        sequence = pd.DataFrame({"number": numbers, "suffix": suffixes, "distance": distances}).drop_duplicates(
            subset="distance", keep="first")["number"].to_list()

    # Remove duplicated addresses.
    sequence = list(OrderedDict.fromkeys(sequence))

    return sequence

def groupby_to_list(df, group_field, list_field):
    """
    Helper function: faster alternative to pandas groupby.apply/agg(list).
    Groups records by one or more fields and compiles an output field into a list for each group.
    """
    
    if isinstance(group_field, list):
        for field in group_field:
            if df[field].dtype.name != "geometry":
                df[field] = df[field].astype("U")
        transpose = df.sort_values(group_field)[[*group_field, list_field]].values.T
        keys, vals = np.column_stack(transpose[:-1]), transpose[-1]
        keys_unique, keys_indexes = np.unique(keys.astype("U") if isinstance(keys, np.object) else keys, 
                                              axis=0, return_index=True)
    
    else:
        keys, vals = df.sort_values(group_field)[[group_field, list_field]].values.T
        keys_unique, keys_indexes = np.unique(keys, return_index=True)
    
    vals_arrays = np.split(vals, keys_indexes[1:])
    
    return pd.Series([list(vals_array) for vals_array in vals_arrays], index=keys_unique).copy(deep=True)

def sort_addresses(numbers, suffixes, distances):
    """
    Sorts the addresses successively by:
    1) distance - the distance of the intersection point along the road segment.
    2) number
    3) suffix
    Taking into account the directionality of the addresses relative to the road segment.
    """

    # Create individual address tuples from separated address components.
    addresses = tuple(zip(numbers, suffixes, distances))

    # Apply initial sorting, by distance, to identify address directionality.
    addresses_sorted = sorted(addresses, key=itemgetter(2))
    directionality = -1 if addresses_sorted[0][0] > addresses_sorted[-1][0] else 1

    # Sort addresses - same direction.
    if directionality == 1:
        return tuple(sorted(addresses, key=itemgetter(2, 1, 0)))

    # Sort addresses - opposite direction.
    else:
        return tuple(sorted(sorted(sorted(
            addresses, key=itemgetter(1), reverse=True),
            key=itemgetter(0), reverse=True),
            key=itemgetter(2)))

In [10]:
# Split address dataframe by parity.
addresses_l = addresses[addresses["parity"] == "l"].copy(deep=True)
addresses_r = addresses[addresses["parity"] == "r"].copy(deep=True)

# Create dataframes from grouped addresses.
cols = ("number", "suffix", "distance")
addresses_l = pd.DataFrame({col: groupby_to_list(addresses_l, "road_index", col) for col in cols})
addresses_r = pd.DataFrame({col: groupby_to_list(addresses_r, "road_index", col) for col in cols})

# Sort addresses.
addresses_l = addresses_l.apply(lambda row: sort_addresses(*row), axis=1)
addresses_r = addresses_r.apply(lambda row: sort_addresses(*row), axis=1)

**Example of a single address grouping, sorted, left and right (index=264):**

In [11]:
# View example address group - left.
for address in addresses_l.loc[addresses_l.index==264].iloc[0]:
    print(address)

(338, '', 127.03100115999284)
(340, '', 241.05817095330403)
(341, 'B', 320.03457447286013)
(341, 'D', 324.8408692908543)
(341, 'E', 326.1533805728225)
(341, 'A', 387.7112313691008)
(343, '', 479.4076579278428)
(347, '', 711.9039993253133)
(350, '', 755.7351224035931)
(352, '', 837.9682009283824)
(349, '', 847.3092809679929)


In [21]:
# View example address group - right.
for address in addresses_r.loc[addresses_r.index==264].iloc[0]:
    print(address)

(337, '', 106.06097642300814)
(339, '', 233.80635153526674)
(342, '', 320.20693093351167)
(341, 'C', 324.6144219630608)
(344, '', 379.9452068085028)
(346, '', 498.629773688345)
(348, '', 576.1215710590544)
(345, '', 579.482713160022)


In [12]:
# Configure addrange attributes.
addrange = pd.DataFrame(index=map(int, {*addresses_l.index, *addresses_r.index}))

# Configure addrange attributes - hnumf, hnuml.
addrange.loc[addresses_l.index, "l_hnumf"] = addresses_l.map(lambda addresses: addresses[0][0])
addrange.loc[addresses_l.index, "l_hnuml"] = addresses_l.map(lambda addresses: addresses[-1][0])
addrange.loc[addresses_r.index, "r_hnumf"] = addresses_r.map(lambda addresses: addresses[0][0])
addrange.loc[addresses_r.index, "r_hnuml"] = addresses_r.map(lambda addresses: addresses[-1][0])

# Configuring addrange attributes - hnumsuff, hnumsufl.
addrange.loc[addresses_l.index, "l_hnumsuff"] = addresses_l.map(lambda addresses: addresses[0][1])
addrange.loc[addresses_l.index, "l_hnumsufl"] = addresses_l.map(lambda addresses: addresses[-1][1])
addrange.loc[addresses_r.index, "r_hnumsuff"] = addresses_r.map(lambda addresses: addresses[0][1])
addrange.loc[addresses_r.index, "r_hnumsufl"] = addresses_r.map(lambda addresses: addresses[-1][1])

# Configuring addrange attributes - hnumtypf, hnumtypl.
addrange.loc[addresses_l.index, "l_hnumtypf"] = addresses_l.map(lambda addresses: "Actual Located")
addrange.loc[addresses_l.index, "l_hnumtypl"] = addresses_l.map(lambda addresses: "Actual Located")
addrange.loc[addresses_r.index, "r_hnumtypf"] = addresses_r.map(lambda addresses: "Actual Located")
addrange.loc[addresses_r.index, "r_hnumtypl"] = addresses_r.map(lambda addresses: "Actual Located")

# Get address number sequence.
address_sequence_l = addresses_l.map(get_number_sequence)
address_sequence_r = addresses_r.map(get_number_sequence)

# Configure addrange attributes - hnumstr.
addrange.loc[addresses_l.index, "l_hnumstr"] = address_sequence_l.map(get_hnumstr)
addrange.loc[addresses_r.index, "r_hnumstr"] = address_sequence_r.map(get_hnumstr)

# Configure addrange attributes - digdirfg.
addrange.loc[addresses_l.index, "l_digdirfg"] = address_sequence_l.map(get_digdirfg)
addrange.loc[addresses_r.index, "r_digdirfg"] = address_sequence_r.map(get_digdirfg)

**Example addrange attributes (index=264):**

In [13]:
# View example address group.
addrange.loc[addrange.index==264].iloc[0]

l_hnumf                      338
l_hnuml                      349
r_hnumf                      337
r_hnuml                      345
l_hnumsuff                      
l_hnumsufl                      
r_hnumsuff                      
r_hnumsufl                      
l_hnumtypf        Actual Located
l_hnumtypl        Actual Located
r_hnumtypf        Actual Located
r_hnumtypl        Actual Located
l_hnumstr              Irregular
r_hnumstr              Irregular
l_digdirfg    Opposite Direction
r_digdirfg    Opposite Direction
Name: 264, dtype: object

## Merge addrange attributes with roads

In [19]:
# Merge addrange attributes with roads.
# roads = roads.merge(addrange, how="left", left_index=True, right_index=True)

roads.loc[roads.index==264].iloc[0]

featurecod                                              RARTLIN
community                                           Yellowknife
routename                                                  None
surface                                                   Paved
date                                                 2015-09-28
roadname                                       OLD AIRPORT ROAD
geometry      LINESTRING Z (-12735876.4681 8967934.843400002...
l_hnumf                                                     338
l_hnuml                                                     349
r_hnumf                                                     337
r_hnuml                                                     345
l_hnumsuff                                                     
l_hnumsufl                                                     
r_hnumsuff                                                     
r_hnumsufl                                                     
l_hnumtypf                              

In [59]:
# Create data for viewing.
intersection_pt_linkages = gpd.GeoDataFrame(
    geometry=addresses.loc[addresses["road_index"]==264][["geometry", "intersection"]].apply(
        lambda row: LineString([pt.coords[0][:2] for pt in row]), axis=1), crs=addresses.crs)
labels = addresses.loc[addresses["road_index"]==264][["number", "suffix", "geometry"]].apply(
    lambda row: (f"{row[0]}{row[1]}", row[2].x, row[2].y), axis=1)

# Configure plot.
fix, ax = plt.subplots(tight_layout=True)
plt.imshow(basemap, extent=extent)
addresses.loc[addresses["road_index"]==264].plot(ax=ax, color="red", label="addresses", markersize=3)
intersection_pt_linkages.plot(ax=ax, color="yellow", label="addresses - intersection point linkages", linewidth=1)
roads.loc[roads.index==264].plot(ax=ax, color="cyan", label="roads", linewidth=1)
ax.ticklabel_format(style="plain")
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha="right")
plt.xlabel("Longitude (m)")
plt.ylabel("Latitude (m)")
plt.title("Example address range (index=244)")
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
for label in labels:
    l, x, y = label
    plt.annotate(l, xy=(x, y), xytext=(x+20, y-20), textcoords="data", ha="center", va="top", fontsize=6, color="red", fontweight="bold", bbox=dict(pad=0.2, fc="black"))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …